In [ ]:
!pip install "git+https://github.com/jemisjoky/TorchMPS.git"
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchmps import MPS

In [ ]:
# Get the data

# Data hyperparameters
nb_train = 200
nb_test = 50
chosen_bs = 100

# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, sampler = train_subset, batch_size=chosen_bs)


test_set = torchvision.datasets.MNIST(root = './datasets', train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, sampler = test_subset, batch_size = chosen_bs)

In [ ]:
# Create and train the FCNN

# Pick the hyperparameters for the FCNN
chosen_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs_fcnn = 1
hidden_size = 70
chosen_loss = nn.CrossEntropyLoss()
#Optimizer parameters
chosen_lr = 0.001
chosen_momentum = 0.9


# Instantiate the fcnn
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.lin1 = nn.Linear(784, hidden_size)
        self.relu1 = nn.ReLU()

        self.lin2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()

        self.lin3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()

        self.lin4 = nn.Linear(hidden_size, hidden_size)
        self.relu4 = nn.ReLU()

        self.lin5 = nn.Linear(hidden_size, hidden_size)
        self.relu5 = nn.ReLU()

        self.lin6 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu1(y)

        y = self.lin2(y)
        y = self.relu2(y)

        y = self.lin3(y)
        y = self.relu3(y)

        y = self.lin4(y)
        y = self.relu4(y)

        y = self.lin5(y)
        y = self.relu5(y)

        y = self.lin6(y)
        return y

#Instantiate and put the model on the chosen device
fcnn_teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
optimizer = torch.optim.Adam(fcnn_teacher.parameters())

#Training loop
for epoch in range(n_epochs_fcnn):
    for (x_mb, y_mb) in train_iterator:

        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        # Foward propagation
        y_hat_mb = fcnn_teacher(x_mb)
        loss = chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(loss.item())


In [ ]:
# Print the accuracy of the fcnn
with torch.no_grad():
    acc_count = 0
    for (x_mb, y_mb) in test_iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        y_mb_hat = fcnn_teacher(x_mb) #predict

        (wedontcare, y_mb_hat_class) = torch.max(y_mb_hat, 1)
        acc_count += (y_mb_hat_class == y_mb).sum().item()

    print("Accuracy: " + str(100* acc_count / len(test_iterator.dataset)))

- Fix non negative values

https://github.com/jemisjoky/TorchMPS/blob/main/train_script.py

he uses the max to get his function!